# Model Manager Examples

## Initialization

By default, ModelManager looks for settings files `"settings.toml"` and/or `".secrets.toml"`, mchat_core uses the excellent [dynaconf](https://www.dynaconf.com/) package for handling settings.  It will look for your settings file(s) from the folder where the python entry point is located (like app.py), then it will look at each parent up to the root.  It will also try looking inside a /config folder at each level.  If your settings are findable, and all you want is to run `ask()` or `aask()`, you can call them directly from the Module

### Normal initialization
```python
from mchat_core import ModuleManager
mm = ModuleManager()
mm.ask("Tell me a joke")
```

### Class Functions `ask()` and `aask()` can be used without instantiation
```python
from mchat_core import ModuleManager as mm
mm.ask("Tell me a joike")
```

### Optional - specify a configuration file

You can point to a specific configuration by passing `settings_files` when instantiating an instance of ModelManager. *Note*: if you want to use a custom configuration, you will need to call `ask()` and `aask()` from your instance of ModuleManager and not directly 

```python
from mchat_core import ModuleManager
mm = ModuleManager(settings_files = ['mycustomsettings.toml'])
mm.ask("Tell me a joke")
```

### Loading setting files (from Dynaconf docs)

Dynaconf will start looking for each file defined in settings_files from the folder where your entry point python file is located (like app.py). Then, it will look at each parent down to the root of the system. For each visited folder, it will also try looking inside a /config folder.

Absolute paths are recognized and dynaconf will attempt to load them directly.
For each file specified in settings_files dynaconf will also try to load an optional name.local.extension. Eg, settings_file="settings.toml" will look for settings.local.toml too.


## Simple`ask()` using default model, `aask()` for an async version

In [1]:
# This requires `settings.toml` to automatically findable, otherwise instantiate
# ModelManager with the correct path to the settings file.
from mchat_core.model_manager import ModelManager

mm = ModelManager()

print(mm.ask("What is the capital of France?"))

# Async version
output = await mm.aask("What is the capital of Spain?")
print(output)

The capital of France is **Paris**.
The capital of Spain is **Madrid**.


## Specifying a particular model and system prompts with `ask()` and `aask()`

In [2]:
from mchat_core.model_manager import ModelManager

mm = ModelManager()
print(mm.ask("What is the capital of Italy?", model="gpt-4o-mini"))

# Viewing the models available
print("\nAvailable chat models:")
print(f"{mm.available_chat_models}\n")

response = await mm.aask("What is the capital of Germany?", system_prompt="respond in pig latin")
print(response)

The capital of Italy is Rome.

Available chat models:
['gpt-4o-mini', 'gpt-4_1-mini', 'gpt-4o', 'gpt-4_1', 'o1', 'o3', 'anthropic-claude-3_5-haiku', 'anthropic-claude-3_7', 'azure_openai_gpt_4o']

Ethay apitalcay ofay ermanyGay isay erlinBay.


## Fully opening a specific model client (returns OpenAIChatCompletionClient or AzureOpenAIChatCompletionClient)

This opens and returns the autogen client using the specified model, additional kwargs are passed to the underlying client


In [3]:
from autogen_core.models import SystemMessage, UserMessage
from mchat_core.model_manager import ModelManager

mm = ModelManager()
client = mm.open_model("gpt-4o-mini")
print(type(client))

out = await client.create([UserMessage(content="What is the capital of Belgium", 
                                       source="user")])
print(out.content)

out = await client.create([SystemMessage(content="Respond in piglatin"), 
                           UserMessage(content="What is the capital of Belgium", 
                                       source="user")])
print(out.content)


<class 'autogen_ext.models.openai._openai_client.OpenAIChatCompletionClient'>
The capital of Belgium is Brussels.
Ethay apitalcay ofway ElgiumBay isway RusselsBay.


# Agent Manager Examples

## Basic Round Robin with two Agents

This example is a `team' of two agents.  One agent is simulating a linux terminal and the second agent is simulating the user of that terminal trying to solve a task.  The flow alternates between the two in the order specifified in the 'team' definition.  

In [9]:
from mchat_core.agent_manager import AutogenManager

agents_yaml = """
linux_computer:
  type: agent
  description: A Linux terminal
  prompt: >
    Act as a Linux terminal. I will type commands and you will reply with
    what the terminal should show. Just return the response, do not put it into a code
    block.  Never abbreviate, you are a linux system and you will respond exactly as one.
    If the command you recieive is not a valid command, you will reply with the 
    appropriate error message.
  extra_context:
    - - human
      - hostname
    - - ai
      - "```shell\nlinux-terminal```"

operator:
  type: agents
  description: An operator of a linux terminal
  prompt: >
    You are an operator of a Linux terminal. You will get a task from a user
    and you will reply with the commands, one at a time, that you want to type into the
    terminal to complete the task.
    Use the information from the terminal output to inform your next command.
    ONLY reply with the command, nothing else. Do not write explanations.
    Your job is to determine the commands and type them, that is it.
    ONLY when you feel you have completed the task, reply with the single
    word TERMINATE.

user_at_terminal:
  type: team
  team_type: round_robin
  description: A user using a linux terminal
  oneshot: false
  max_rounds: 11
  termination_message: TERMINATE
  agents:
    - operator
    - linux_computer
"""

print("Example 1: Live output via callback")
print("-----------------------------------\n")

# callback to output messages to the terminal
async def show_token(message, **kwargs):
    print(f"{kwargs['agent']}:\n{message}\n\n", flush=True)

# Load the agents
am = AutogenManager(message_callback=show_token, agent_paths=[agents_yaml])

# Choose the agent to run
await am.new_conversation("user_at_terminal")

# Start the conversation - this will stream output to the terminal via the callback
out = await am.ask("determine if this computer has been compromised by an attacker")

# Example 2 - No live output, collect the messages from the TaskResult
print("Example 2: Collect output from TaskResult")
print("----------------------------------------\n")
am2 = AutogenManager(agent_paths=[agents_yaml])
await am2.new_conversation("user_at_terminal")
out = await am2.ask("erase all activity logs on this computer")

for msg in out.messages:
    print(f"{msg.source}:\n{msg.content}\n\n")


Example 1: Live output via callback
-----------------------------------

operator:
lastlog | grep -v "Never logged in"


linux_computer:
Username         Port     From             Latest
root             pts/0    192.168.1.20     Wed Jun 12 10:15:33 +0000 2024
user             pts/1    10.0.2.15        Tue Jun 11 08:22:05 +0000 2024


operator:
w


linux_computer:
 10:36:50 up  2:55,  2 users,  load average: 0.03, 0.04, 0.01
USER     TTY      FROM             LOGIN@   IDLE   JCPU   PCPU WHAT
root     pts/0    192.168.1.20     10:15    0.00s  0.20s  0.02s w
user     pts/1    10.0.2.15        08:22    1:30m  0.01s  0.01s -bash


operator:
whoami


linux_computer:
operator


operator:
ps aux --sort=-%cpu | head -n 15


linux_computer:
USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root        1123  2.3  0.8 168400  7424 pts/0    Ss   10:15   0:01 /usr/bin/python3 /usr/bin/somecpuintensiveprocess
root        1022  1.1  0.5 105600  4888 pts/0    Ss   10:16   0:00 